<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [1]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


# Define league settings

In [2]:
year = 2024
num_rounds = 15
num_teams = 12
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

## Load player projections

In [3]:
weekly_projections_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/projection_data/2024/weekly_projections_{year}.csv'
weekly_projections = pd.read_csv(weekly_projections_url)
print(weekly_projections.columns)
weekly_projections_colums = ['player', 'position', 'team', 'points', 'week', 'year']
weekly_projections = (
    weekly_projections[weekly_projections_colums]
    .rename(columns={'player': 'Player', 'position': 'POS'})
    .query('POS in @allowed_positions')
    .copy()
)

print(len(weekly_projections))
print(weekly_projections.head(10))

Index(['Unnamed: 0', 'player', 'position', 'team', 'points', 'sd_pts',
       'dropoff', 'floor', 'ceiling', 'points_vor', 'floor_vor', 'ceiling_vor',
       'rank', 'floor_rank', 'ceiling_rank', 'position_rank', 'tier', 'adp',
       'aav', 'uncertainty', 'week', 'year'],
      dtype='object')
4982
                Player POS team  points  week  year
0  Christian McCaffrey  RB   SF    21.4     1  2024
1          Tyreek Hill  WR  MIA    21.5     1  2024
2    Amon-Ra St. Brown  WR  DET    19.8     1  2024
3           Josh Allen  QB  BUF    24.6     1  2024
4       Bijan Robinson  RB  ATL    16.9     1  2024
5          Breece Hall  RB  NYJ    16.9     1  2024
6           Nick Chubb  RB  CLE    16.3     1  2024
7       Saquon Barkley  RB  PHI    16.3     1  2024
8     Justin Jefferson  WR  MIN    18.5     1  2024
9         Jahmyr Gibbs  RB  DET    16.0     1  2024


In [4]:
f = weekly_projections.pivot_table(
    index=['Player', 'POS'],
    columns='week',
    values='points',
    aggfunc='first'
)

# Rename columns to 'Week X'
f.columns = [f'Week_{col}' for col in f.columns]
f['TTL'] = f.sum(axis=1)
f = f.reset_index()
f = f.fillna(0)

f = f.sort_values(['Player', 'TTL'], ascending=[True, False])
f = f.drop_duplicates(subset=['Player'], keep='first').reset_index(drop=True)

# calculate dropoff inside grouped POS
f['dropoff'] = (f.sort_values(['POS','TTL'], ascending=[True, False]).groupby('POS')['TTL'].diff(-1).fillna(0.0))
dropoff_weight = {'QB': 1.0, 'RB': 1.0, 'WR': 1.0, 'TE': 0.9, 'K': 0.4, 'DST': 0.3}
f['dropoff'] = f.apply(lambda row: row['dropoff'] * dropoff_weight.get(row['POS'], 1.0), axis=1)

# calculate value-over-replacement (VOR)
vor_dict = {}
vor_weight = {'QB': 0.8, 'RB': 1.0, 'WR': 1.0, 'TE': 0.8, 'K': 0.25, 'DST': 0.25}
for pos, limit in lineup_req.items():
    replacement_index = limit * num_teams - 1
    pos_df = f[f['POS'] == pos].sort_values('TTL', ascending=False).reset_index(drop=True)
    if len(pos_df) > replacement_index:
        replacement_ttl = pos_df.loc[replacement_index, 'TTL']
    else:
        replacement_ttl = 0

    weight = vor_weight.get(pos, 1.0)

    for idx, row in pos_df.iterrows():
        vor = (row['TTL'] - replacement_ttl) * weight
        vor_dict[row['Player']] = row['TTL'] - replacement_ttl


f = f.sort_values('TTL', ascending=False)
f['VOR'] = f['Player'].map(vor_dict)
head_size = 15
print(len(f))
f.head(head_size)

476


,Player,POS,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,...,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,Week_18,TTL,dropoff,VOR
309,Lamar Jackson,QB,19.0,21.5,20.0,19.8,21.0,22.9,21.7,21.8,...,21.9,21.4,0.0,22.7,22.3,22.3,23.6,368.7,9.1,88.68
263,Josh Allen,QB,24.6,22.8,22.9,22.0,21.6,18.7,21.6,21.9,...,0.0,22.1,22.8,23.2,25.8,24.7,0.0,359.6,20.6,79.58
235,Jayden Daniels,QB,16.7,18.7,17.5,20.9,20.8,21.0,22.6,19.2,...,20.3,19.0,0.0,20.6,18.4,20.9,21.4,339.0,9.3,58.98
201,Ja'Marr Chase,WR,16.3,17.4,19.4,18.3,17.2,19.3,18.4,20.2,...,0.0,20.2,20.7,21.5,22.4,21.9,22.6,336.6,12.4,133.20
246,Joe Burrow,QB,18.4,17.9,19.4,18.7,18.1,19.2,18.1,18.7,...,0.0,19.9,21.3,20.3,21.5,20.0,20.4,329.7,9.3,49.68
278,Justin Jefferson,WR,18.5,17.6,19.4,19.0,18.0,0.0,19.9,19.2,...,17.9,17.7,18.9,18.6,18.3,20.0,21.8,324.2,14.8,120.80
219,Jalen Hurts,QB,21.8,22.7,21.2,20.6,0.0,20.7,19.4,19.7,...,21.5,21.5,22.5,20.7,21.9,0.0,0.0,320.4,15.5,40.38
32,Bijan Robinson,RB,16.9,19.4,17.6,17.5,16.0,16.0,17.5,17.8,...,0.0,18.8,17.7,19.3,20.3,18.2,21.1,311.2,11.6,122.22
16,Amon-Ra St. Brown,WR,19.8,18.6,17.9,16.9,0.0,17.7,16.7,18.8,...,18.7,18.2,17.7,17.8,19.2,18.4,20.1,309.4,37.4,106.00
306,Kyler Murray,QB,17.4,18.8,19.5,20.9,17.1,17.7,17.6,17.6,...,17.6,16.8,17.3,18.6,18.7,17.2,17.4,304.9,2.6,24.88


In [84]:
positions = ['QB', 'RB', 'WR', 'TE', 'K', 'DST']
sampled_players = []

for pos in positions:
    top_50 = f[f['POS'] == pos] \
        .sort_values('TTL', ascending=False) \
        .head(20)

    if not top_50.empty:
        sampled_players.append(top_50.sample(1))

# Ergebnisse kombinieren
sample_df = pd.concat(sampled_players).reset_index(drop=True)

# Spaltenauswahl (optional)
sample_df = sample_df[['Player', 'POS', 'Week_1', 'Week_2', 'Week_17', 'TTL', 'dropoff', 'VOR']]

print(sample_df)


              Player  POS  Week_1  Week_2  Week_17     TTL  dropoff     VOR
0        Jalen Hurts   QB   21.80   22.70     0.00  320.40   15.500   40.38
1      D'Andre Swift   RB   10.50   11.10    12.40  220.36    5.620   31.38
2  Amon-Ra St. Brown   WR   19.80   18.60    18.40  309.40   37.400  106.00
3        Jonnu Smith   TE    5.91    4.90    12.30  143.68    1.359    7.86
4       Cairo Santos    K    8.25    7.63     7.73  131.84    0.272   -2.42
5              Lions  DST    4.78    4.86     4.42   85.48    0.015    1.54


In [5]:
duplicates = f[f.duplicated(subset=['Player'], keep=False)]

if duplicates.empty:
    print("No duplicates existing")
else:
    print(duplicates.sort_values('Player'))

No duplicates existing


## Create Player dataset

In [6]:
players = f[['Player', 'POS', 'TTL']].copy()
players = players.rename(columns={
    'Player': 'Player',
    'POS': 'POS',
    'TTL': 'TTL'
})
players = players.sort_values('TTL', ascending=False).reset_index(drop=True)
players['Rank'] = players.index + 1

players.info()
print(len(players))
print(players)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  476 non-null    object 
 1   POS     476 non-null    object 
 2   TTL     476 non-null    float64
 3   Rank    476 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 15.0+ KB
476
             Player POS     TTL  Rank
0     Lamar Jackson  QB  368.70     1
1        Josh Allen  QB  359.60     2
2    Jayden Daniels  QB  339.00     3
3     Ja'Marr Chase  WR  336.60     4
4        Joe Burrow  QB  329.70     5
..              ...  ..     ...   ...
471        C.J. Ham  RB    1.78   472
472  Frank Gore Jr.  RB    1.74   473
473  Kendall Milton  RB    1.62   474
474  Reggie Gilliam  RB    1.53   475
475       Jake Funk  RB    1.45   476

[476 rows x 4 columns]


### discarded

In [7]:
# adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_{year}_Overall_ADP_Rankings.csv'
# players_adp = pd.read_csv(adp_url)#, on_bad_lines='skip')
# players_adp['POS'] = players_adp['POS'].str.replace('\d+', '', regex=True)

# def extract_numbers(s):
#     if pd.isna(s):
#         return None
#     numbers = re.findall(r'\d+', str(s))
#     if numbers:
#         return int(numbers[0])
#     return None

# num_of_players = len(players_adp)

# players_adp = players_adp[['Player', 'Bye', 'POS', 'AVG']].copy()
# players_adp['Bye'] = players_adp['Bye'].apply(extract_numbers)
# players_adp['Bye'] = players_adp['Bye'].fillna(0).astype(int)

# players_adp.info()
# print(players_adp.value_counts('POS'))
# print(players_adp.head(20))

# Initiate Model and apply Optimization

In [8]:
# ==========================
# 1. PARAMETERS
# ==========================

players_list = players['Player'].copy().tolist()
positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
weeks = list(range(1, 18))
pos = dict(zip(weekly_projections['Player'], weekly_projections['POS']))
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
week_cols = [col for col in f.columns if col.startswith('Week_')]

# Build f_dict: weekly projections and dropoff for each player
f_dict = {
    row['Player']: {
        **{int(week.replace('Week_', '')): row[week] for week in week_cols},
        'dropoff': row['dropoff'],
        'VOR': row['VOR']
    }
    for _, row in f.iterrows()
}

beta = {t: 100.0 for t in weeks}
gamma = {'QB': 2, 'RB': 3, 'WR': 3, 'TE': 2, 'K': 1, 'DST': 1}
alpha, lambda_0, lambda_1, lambda_2, lambda_3, lambda_4 = 1.0, 1, 3, 4, 0.5, 400
df_sorted = players.sort_values('Rank').reset_index(drop=True)
topk_pct = 0.0025

# ==========================
# 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# ==========================

teams = [f'Team {i+1}' for i in range(num_teams)]
DM_team = 'Team 1'

# Snake draft order
draft_order = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
    # 1) Sort remaining players by rank
    remaining_players = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(remaining_players) * topk_pct))

    # 2) Calculate deficits per position (min requirement minus current roster)
    deficits = {
        j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in lineup_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) If deficits exist, pick from candidates in those positions
    if needed:
        candidates = [p for p in remaining_players if pos[p] in needed]
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: pick randomly from top-k overall
    return random.choice(remaining_players[:topk])

# ==========================
# 3. DRAFT INITIALIZATION
# ==========================

rosters = {tm: [] for tm in teams}
available = set(players_list)
draft_log = []

# ==========================
# 4. MAIN DRAFT LOOP
# ==========================

for pick_idx, team in enumerate(draft_order, start=1):

    # ---- 4.1: Update remaining player ranking ----
    for p in available:
      ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
      if len(ranks) != 1:
          print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

    remaining_players = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, 'Rank'].item())
    Rk = {p: i+1 for i, p in enumerate(remaining_players)}
    player_vars = set(remaining_players) | set(rosters[team])
    picks_remaining = num_rounds - len(rosters[team])

    # ---- 4.2: DM-Team (your team) picks via MIP ----
    if team == DM_team:
        m = Model(sense=maximize, solver_name='CBC')

        # --- Decision variables ---
        # y[i]: 1 if player i is drafted by DM, 0 otherwise
        y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in player_vars}
        # x[i, t]: share of player i's points used in week t (continuous)
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
        # z[t]: 1 if DM wins in week t, 0 otherwise
        z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

        # --- Objective function ---
        m.objective = (
            lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in remaining_players for t in weeks)
            + lambda_1 * xsum(z[t] for t in weeks[:15])
            + lambda_2 * xsum(z[t] for t in weeks[15:])
            + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in remaining_players)
            - lambda_4 * xsum(f_dict[i]['VOR'] * y[i] for i in remaining_players)
        )

        # --- Constraints ---
        # Fix previous picks (already drafted players must remain picked)
        for p in rosters[team]:
            m += y[p] == 1

        # Enforce that exactly the remaining picks are made
        m += xsum(y[i] for i in remaining_players) == picks_remaining

        # Enforce minimum requirements for each position (relative to already drafted players)
        for pos_name, req in lineup_req.items():
            already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
            need = max(0, req - already_satisfied)
            m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

        # Weekly lineup constraints and position limits
        for j in positions:
            # m += xsum(y[i] for i in remaining_players if pos[i] == j) >= gamma[j], f'{j}_min_req'  # Minimum number per position
            for t in weeks:
                m += xsum(x[i, t] for i in remaining_players if pos[i] == j) <= lineup_req[j]  # Weekly lineup limit

        # Only drafted players can be in the weekly lineup
        for i in remaining_players:
            for t in weeks:
                m += x[i, t] <= y[i]

        # Win indicator constraints
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

        # Robust draft constraint (to simulate uncertainty in opponent picks)
        n_k = pick_idx
        for future_pick in range(
            pick_idx + 1,
            pick_idx + picks_remaining * len(teams),
            len(teams)
        ):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        # --- Solve the MIP model ---
        m.optimize()
        # Check for infeasibility
        if m.num_solutions == 0:
            raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

        print(f"\n--- DM Pick {pick_idx} ---")
        for i in remaining_players:
            if y[i].x is not None and y[i].x >= 0.9:
                ttl = sum(f_dict[i][t] for t in weeks)
                vor = vor_dict.get(i, 0.0)
                print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")


        # Extract chosen player for this pick
        chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f'No feasible pick at {pick_idx}')
        pick = min(chosen, key=lambda i: Rk[i])

    # ---- 4.3: Opponent pick (simple heuristic) ----
    else:
        current_round = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            lineup_req=lineup_req,
            topk_pct=dynamic_topk_pct
        )

    # ---- 4.4: Update rosters and draft log ----
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Pick': pick_idx,
        'Team': team,
        'Player': pick,
        'Round': (pick_idx - 1) // len(teams) + 1,
        'POS': pos[pick]
    })

# ==========================
# 5. CREATE DRAFT DATAFRAME
# ==========================

df_draft = pd.DataFrame(draft_log)



--- DM Pick 1 ---
Ja'Marr Chase             (WR)  TTL= 314.0   VOR= 133.2
Devon Achane              (RB)  TTL= 269.8   VOR=  95.5
Tyreek Hill               (WR)  TTL= 256.3   VOR=  65.6
Brock Bowers              (TE)  TTL= 205.0   VOR=  82.9
Travis Kelce              (TE)  TTL= 217.9   VOR=  82.1
Trey McBride              (TE)  TTL= 194.3   VOR=  73.0
David Montgomery          (RB)  TTL= 189.4   VOR=   0.4
Sam LaPorta               (TE)  TTL= 156.8   VOR=  33.0
Justin Tucker             (K)  TTL= 149.8   VOR=  24.8
Kaimi Fairbairn           (K)  TTL= 144.0   VOR=  18.2
Tyler Bass                (K)  TTL= 138.6   VOR=  13.3
Vikings                   (DST)  TTL=  86.9   VOR=   7.3
Texans                    (DST)  TTL=  84.3   VOR=   6.0
Broncos                   (DST)  TTL=  83.6   VOR=   5.7
Taysom Hill               (QB)  TTL=  48.6   VOR= -87.2

--- DM Pick 24 ---
Alvin Kamara              (RB)  TTL= 259.4   VOR=  81.4
Aaron Jones               (RB)  TTL= 227.5   VOR=  52.6
Brock Bow

In [87]:
# mit flex:
# ==========================
# 1. PARAMETERS
# ==========================

players_list = players['Player'].copy().tolist()
positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
flex_eligible = {'RB', 'WR', 'TE'}
weeks = list(range(1, 18))
pos = dict(zip(weekly_projections['Player'], weekly_projections['POS']))
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
week_cols = [col for col in f.columns if col.startswith('Week_')]

# Build f_dict: weekly projections and dropoff for each player
f_dict = {
    row['Player']: {
        **{int(week.replace('Week_', '')): row[week] for week in week_cols},
        'dropoff': row['dropoff'],
        'VOR': row['VOR']
    }
    for _, row in f.iterrows()
}

beta = {t: 100.0 for t in weeks}
gamma = {'QB': 2, 'RB': 3, 'WR': 3, 'TE': 2, 'K': 1, 'DST': 1}
alpha, lambda_0, lambda_1, lambda_2, lambda_3, lambda_4 = 1.0, 1, 3, 4, 0.5, 400
df_sorted = players.sort_values('Rank').reset_index(drop=True)
topk_pct = 0.0025

# ==========================
# 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# ==========================

teams = [f'Team {i+1}' for i in range(num_teams)]
DM_team = 'Team 1'

# Snake draft order
draft_order = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
    remaining_players = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(remaining_players) * topk_pct))

    deficits = {
        j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in lineup_req if j != 'FLEX'
    }
    needed = [j for j, d in deficits.items() if d > 0]

    if needed:
        candidates = [p for p in remaining_players if pos[p] in needed]
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    return random.choice(remaining_players[:topk])

# ==========================
# 3. DRAFT INITIALIZATION
# ==========================

rosters = {tm: [] for tm in teams}
available = set(players_list)
draft_log = []

# ==========================
# 4. MAIN DRAFT LOOP
# ==========================

for pick_idx, team in enumerate(draft_order, start=1):

    # ---- 4.1: Update remaining player ranking ----
    for p in available:
        ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
        if len(ranks) != 1:
            print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

    remaining_players = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, 'Rank'].item())
    Rk = {p: i+1 for i, p in enumerate(remaining_players)}
    player_vars = set(remaining_players) | set(rosters[team])
    picks_remaining = num_rounds - len(rosters[team])

    # ---- 4.2: DM-Team (your team) picks via MIP ----
    if team == DM_team:
        m = Model(sense=maximize, solver_name='CBC')

        # --- Decision variables ---
        y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in player_vars}
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

        # --- Objective function ---
        m.objective = (
            lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in remaining_players for t in weeks)
            + lambda_1 * xsum(z[t] for t in weeks[:15])
            + lambda_2 * xsum(z[t] for t in weeks[15:])
            + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in remaining_players)
            - lambda_4 * xsum(f_dict[i]['VOR'] * y[i] for i in remaining_players)
        )

        # --- Constraints ---
        for p in rosters[team]:
            m += y[p] == 1

        m += xsum(y[i] for i in remaining_players) == picks_remaining

        for pos_name, req in lineup_req.items():
            if pos_name != 'FLEX':
                already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
                need = max(0, req - already_satisfied)
                m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

        for t in weeks:
            # Standard-Positionen (ohne FLEX)
            for j in positions - {'FLEX'}:
                m += xsum(x[i, t] for i in remaining_players if pos[i] == j) <= lineup_req.get(j, 0)
            # FLEX-Position: Summe der eligible Spieler
            m += xsum(x[i, t] for i in remaining_players if pos[i] in flex_eligible) <= lineup_req['FLEX']

            # Kein Spieler darf in einer Woche mehr als einen Slot belegen (Stamm + FLEX)
            for i in remaining_players:
                m += (
                    (x[i, t] if pos[i] in positions - {'FLEX'} else 0) +
                    (x[i, t] if pos[i] in flex_eligible else 0)
                    <= 1
                )

        for i in remaining_players:
            for t in weeks:
                m += x[i, t] <= y[i]

        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

        n_k = pick_idx
        for future_pick in range(
            pick_idx + 1,
            pick_idx + picks_remaining * len(teams),
            len(teams)
        ):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        # --- Solve the MIP model ---
        m.optimize()
        if m.num_solutions == 0:
            raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

        print(f"\n--- DM Pick {pick_idx} ---")
        for i in remaining_players:
            if y[i].x is not None and y[i].x >= 0.9:
                ttl = sum(f_dict[i][t] for t in weeks)
                vor = vor_dict.get(i, 0.0)
                print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")

        chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f'No feasible pick at {pick_idx}')
        pick = min(chosen, key=lambda i: Rk[i])

    # ---- 4.3: Opponent pick (simple heuristic) ----
    else:
        current_round = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            lineup_req=lineup_req,
            topk_pct=dynamic_topk_pct
        )

    # ---- 4.4: Update rosters and draft log ----
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Pick': pick_idx,
        'Team': team,
        'Player': pick,
        'Round': (pick_idx - 1) // len(teams) + 1,
        'POS': pos[pick]
    })

# ==========================
# 5. CREATE DRAFT DATAFRAME
# ==========================

df_draft = pd.DataFrame(draft_log)



--- DM Pick 1 ---
Ja'Marr Chase             (WR)  TTL= 314.0   VOR= 133.2
Devon Achane              (RB)  TTL= 269.8   VOR=  95.5
Tyreek Hill               (WR)  TTL= 256.3   VOR=  65.6
Brock Bowers              (TE)  TTL= 205.0   VOR=  82.9
Travis Kelce              (TE)  TTL= 217.9   VOR=  82.1
Trey McBride              (TE)  TTL= 194.3   VOR=  73.0
David Montgomery          (RB)  TTL= 189.4   VOR=   0.4
Sam LaPorta               (TE)  TTL= 156.8   VOR=  33.0
Justin Tucker             (K)  TTL= 149.8   VOR=  24.8
Kaimi Fairbairn           (K)  TTL= 144.0   VOR=  18.2
Tyler Bass                (K)  TTL= 138.6   VOR=  13.3
Vikings                   (DST)  TTL=  86.9   VOR=   7.3
Texans                    (DST)  TTL=  84.3   VOR=   6.0
Broncos                   (DST)  TTL=  83.6   VOR=   5.7
Taysom Hill               (QB)  TTL=  48.6   VOR= -87.2

--- DM Pick 24 ---
Josh Jacobs               (RB)  TTL= 242.7   VOR=  71.5
Aaron Jones               (RB)  TTL= 227.5   VOR=  52.6
Brock Bow

In [88]:
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)

OptimizationStatus.OPTIMAL


## Inspect results

In [89]:
merged = pd.merge(df_draft, f[['Player', 'TTL']], on='Player', how='left')

# Replace missing TTLs (e.g., für Spieler ohne Projection) mit 0
merged['TTL'] = merged['TTL'].fillna(0)

# Group by Team and sum TTL to get total projection per team
team_ttl_proj = merged.groupby('Team')['TTL'].sum().reset_index()
team_ttl_proj = team_ttl_proj.rename(columns={'TTL': 'TTL_proj'})
print(team_ttl_proj)

position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
print(position_counts_df)

result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df["Pick Info"] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]

for team, df in result_dfs.items():
    print(f'=== {team} ===')
    print(df)
    print()

       Team  TTL_proj
0    Team 1   2859.09
1   Team 10   2758.31
2   Team 11   2857.38
3   Team 12   2794.20
4    Team 2   2994.84
5    Team 3   2843.16
6    Team 4   2624.62
7    Team 5   2670.77
8    Team 6   2767.72
9    Team 7   2716.01
10   Team 8   2829.38
11   Team 9   2835.24
POS      DST  K  QB  RB  TE  WR
Team                           
Team 1     3  1   1   4   3   3
Team 10    1  2   3   3   1   5
Team 11    1  2   2   3   1   6
Team 12    1  2   3   5   2   2
Team 2     1  1   3   2   1   7
Team 3     1  2   4   3   1   4
Team 4     1  3   3   2   2   4
Team 5     1  2   3   3   3   3
Team 6     1  3   2   2   1   6
Team 7     1  1   2   3   1   7
Team 8     1  3   3   2   3   3
Team 9     1  1   2   5   1   5
=== Team 1 ===
                  Player          Pick Info  POS
0          Ja'Marr Chase     Round 1 Pick 1   WR
23           Josh Jacobs    Round 2 Pick 24   RB
24          Drake London    Round 3 Pick 25   WR
47           C.J. Stroud    Round 4 Pick 48   QB
48    

# Evaluation

## merge df_draft with f

In [11]:
roster_projections = df_draft.merge(f, on='Player', how='left')

# Optional: Sortieren nach Team und Pick
roster_projections = roster_projections.sort_values(by=['Team', 'Pick']).reset_index(drop=True)
roster_projections

,Pick,Team,Player,Round,POS_x,POS_y,Week_1,Week_2,Week_3,Week_4,...,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,Week_18,TTL,dropoff,VOR
0,1,Team 1,Ja'Marr Chase,1,WR,WR,16.30,17.40,19.40,18.30,...,0.00,20.20,20.70,21.50,22.40,21.90,22.60,336.60,12.400,133.20
1,24,Team 1,Alvin Kamara,2,RB,RB,15.50,14.80,19.20,17.90,...,0.00,19.60,21.90,16.50,0.00,5.28,11.00,270.38,3.580,81.40
2,25,Team 1,Tyreek Hill,3,WR,WR,21.50,21.60,15.90,14.50,...,15.20,14.10,14.10,15.80,15.70,14.20,12.70,269.00,11.000,65.60
3,48,Team 1,Sam Darnold,4,QB,QB,13.40,13.50,14.40,15.20,...,15.30,17.60,18.30,18.60,17.30,18.20,20.50,283.00,2.980,2.98
4,49,Team 1,Tony Pollard,5,RB,RB,11.00,12.00,13.70,12.50,...,13.40,13.50,14.40,14.10,12.90,12.30,11.30,224.00,1.300,35.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,129,Team 9,Tyler Bass,11,K,K,9.35,8.57,8.97,8.67,...,0.00,9.30,9.19,8.24,9.39,9.32,8.92,147.56,0.540,13.30
176,136,Team 9,Josh Downs,12,WR,WR,0.00,0.00,7.24,7.73,...,12.20,0.00,0.00,11.10,11.10,11.30,12.60,153.76,0.300,-49.64
177,153,Team 9,Dalton Kincaid,13,TE,TE,11.90,10.60,10.20,10.20,...,0.00,0.00,0.00,9.25,9.71,9.57,4.83,135.82,0.162,0.00
178,160,Team 9,Jaylen Warren,14,RB,RB,9.83,8.71,8.82,0.00,...,8.90,10.20,9.36,9.19,8.81,10.40,11.60,134.86,5.670,-54.12


# create schedule

In [12]:
num_reg_weeks = 14
playoff_weeks = [15, 16, 17]

def create_reg_schedule(teams):
    n = len(teams)
    schedule = []
    for week in range(num_reg_weeks):
        week_matches = []
        for i in range(n//2):
            team1 = teams[i]
            team2 = teams[n-1-i]
            week_matches.append((team1, team2))
        schedule.append(week_matches)
        # Rotate teams except the first one
        teams = [teams[0]] + [teams[-1]] + teams[1:-1]
    return schedule

reg_schedule = create_reg_schedule(teams)
reg_schedule

[[('Team 1', 'Team 12'),
  ('Team 2', 'Team 11'),
  ('Team 3', 'Team 10'),
  ('Team 4', 'Team 9'),
  ('Team 5', 'Team 8'),
  ('Team 6', 'Team 7')],
 [('Team 1', 'Team 11'),
  ('Team 12', 'Team 10'),
  ('Team 2', 'Team 9'),
  ('Team 3', 'Team 8'),
  ('Team 4', 'Team 7'),
  ('Team 5', 'Team 6')],
 [('Team 1', 'Team 10'),
  ('Team 11', 'Team 9'),
  ('Team 12', 'Team 8'),
  ('Team 2', 'Team 7'),
  ('Team 3', 'Team 6'),
  ('Team 4', 'Team 5')],
 [('Team 1', 'Team 9'),
  ('Team 10', 'Team 8'),
  ('Team 11', 'Team 7'),
  ('Team 12', 'Team 6'),
  ('Team 2', 'Team 5'),
  ('Team 3', 'Team 4')],
 [('Team 1', 'Team 8'),
  ('Team 9', 'Team 7'),
  ('Team 10', 'Team 6'),
  ('Team 11', 'Team 5'),
  ('Team 12', 'Team 4'),
  ('Team 2', 'Team 3')],
 [('Team 1', 'Team 7'),
  ('Team 8', 'Team 6'),
  ('Team 9', 'Team 5'),
  ('Team 10', 'Team 4'),
  ('Team 11', 'Team 3'),
  ('Team 12', 'Team 2')],
 [('Team 1', 'Team 6'),
  ('Team 7', 'Team 5'),
  ('Team 8', 'Team 4'),
  ('Team 9', 'Team 3'),
  ('Team 10', 'T

## Load real life player data

In [13]:
data_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/Weekly_Data/weekly_data_{year}.csv'
weekly_data = pd.read_csv(data_url)
weekly_data

,player_display_name,position,season,week,fantasy_points_ppr
0,Aaron Rodgers,QB,2024,1,8.58
1,Aaron Rodgers,QB,2024,2,15.14
2,Aaron Rodgers,QB,2024,3,21.04
3,Aaron Rodgers,QB,2024,4,11.60
4,Aaron Rodgers,QB,2024,5,11.76
...,...,...,...,...,...
5592,Trey Benson,RB,2024,10,10.70
5593,Trey Benson,RB,2024,12,1.80
5594,Trey Benson,RB,2024,13,2.00
5595,Trey Benson,RB,2024,14,2.90


In [14]:
# Roster limits
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

# Für jede Woche und jedes Team die beste Aufstellung bestimmen
def get_best_lineup(team, week, roster_projections, lineup_req):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team]
    lineup = []
    for pos, limit in lineup_req.items():
        candidates = team_roster[team_roster['POS_x'] == pos]
        # Sortiere nach projection für die Woche, nimm die besten N
        starters = candidates.sort_values(week_col, ascending=False).head(limit)
        lineup.append(starters)
    return pd.concat(lineup)

# Punkte aus weekly_data holen
def get_actual_points(lineup, week, weekly_data):
    merged = lineup.merge(
        weekly_data[weekly_data['week'] == week],
        left_on='Player', right_on='player_display_name', how='left'
    )
    # Fülle fehlende Werte (z.B. bei Bye Weeks) mit 0
    merged['fantasy_points_ppr'] = merged['fantasy_points_ppr'].fillna(0)
    return merged['fantasy_points_ppr'].sum()

# Für alle Wochen und alle Matchups durchlaufen
results = []
for week_idx, matchups in enumerate(reg_schedule, 1):
    for team1, team2 in matchups:
        lineup1 = get_best_lineup(team1, week_idx, roster_projections, lineup_req)
        lineup2 = get_best_lineup(team2, week_idx, roster_projections, lineup_req)
        points1 = get_actual_points(lineup1, week_idx, weekly_data)
        points2 = get_actual_points(lineup2, week_idx, weekly_data)
        winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
        loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
        results.append({
            'Woche': week_idx,
            'Team 1': team1,
            'Team 2': team2,
            'Punkte Team 1': points1,
            'Punkte Team 2': points2,
            'Sieger': winner,
            'Verlierer': loser
        })
df_results = pd.DataFrame(results)
df_results

,Woche,Team 1,Team 2,Punkte Team 1,Punkte Team 2,Sieger,Verlierer
0,1,Team 1,Team 12,89.02,50.02,Team 1,Team 12
1,1,Team 2,Team 11,79.32,101.74,Team 11,Team 2
2,1,Team 3,Team 10,54.18,107.20,Team 10,Team 3
3,1,Team 4,Team 9,64.86,70.84,Team 9,Team 4
4,1,Team 5,Team 8,82.16,62.68,Team 5,Team 8
...,...,...,...,...,...,...,...
79,14,Team 11,Team 9,68.00,70.62,Team 9,Team 11
80,14,Team 12,Team 8,86.16,89.80,Team 8,Team 12
81,14,Team 2,Team 7,70.34,108.82,Team 7,Team 2
82,14,Team 3,Team 6,104.88,114.16,Team 6,Team 3


In [15]:
# Deine vorhandene Liste mit Teams
teams = [f'Team {i+1}' for i in range(num_teams)]

# Sieger und Verlierer zählen (Unentschieden ausschließen)
wins = df_results[df_results['Sieger'] != 'Unentschieden']['Sieger'].value_counts()
losses = df_results[df_results['Verlierer'] != 'Unentschieden']['Verlierer'].value_counts()

# Draws zählen: alle Teams, die in einem Unentschieden beteiligt waren
draws = (
    df_results[df_results['Sieger'] == 'Unentschieden'][['Team 1', 'Team 2']]
    .stack()
    .value_counts()
)

points_for = pd.concat([
    df_results[['Team 1', 'Punkte Team 1']].rename(columns={'Team 1': 'Team', 'Punkte Team 1': 'Points'}),
    df_results[['Team 2', 'Punkte Team 2']].rename(columns={'Team 2': 'Team', 'Punkte Team 2': 'Points'})
])
points_for = points_for.groupby('Team')['Points'].sum()

# Punkte, die jedes Team kassiert hat ("Points Against")
points_against = pd.concat([
    df_results[['Team 1', 'Punkte Team 2']].rename(columns={'Team 1': 'Team', 'Punkte Team 2': 'Points'}),
    df_results[['Team 2', 'Punkte Team 1']].rename(columns={'Team 2': 'Team', 'Punkte Team 1': 'Points'})
])
points_against = points_against.groupby('Team')['Points'].sum()

# Zusammenführen in ein DataFrame
record = pd.DataFrame({'Team': teams})
record['Wins'] = record['Team'].map(wins).fillna(0).astype(int)
record['Losses'] = record['Team'].map(losses).fillna(0).astype(int)
record['Draws'] = record['Team'].map(draws).fillna(0).astype(int)
record['Points For'] = record['Team'].map(points_for).fillna(0)
record['Points Against'] = record['Team'].map(points_against).fillna(0)
record = record.sort_values(by=['Wins', 'Draws', 'Points For'], ascending=[False, False, False]).reset_index(drop=True)

record

,Team,Wins,Losses,Draws,Points For,Points Against
0,Team 1,12,2,0,1392.46,1074.64
1,Team 7,9,5,0,1217.88,1085.14
2,Team 4,8,6,0,1241.20,1214.66
3,Team 6,8,6,0,1236.14,1287.82
4,Team 9,8,6,0,1225.06,1124.76
5,Team 11,7,7,0,1269.80,1168.18
6,Team 2,6,8,0,1230.14,1216.50
7,Team 5,6,8,0,1139.34,1218.62
8,Team 8,6,8,0,1034.66,1168.26
9,Team 10,5,9,0,1171.30,1295.86


In [16]:
# Setze die Playoff-Wochen
playoff_weeks = [15, 16, 17]

# Teams nach Rang sortieren (wie zuvor)
ranked_teams = record['Team'].tolist()

# Woche 15: Seed 3 vs 6, Seed 4 vs 5
week_15_matchups = [
    (ranked_teams[2], ranked_teams[5]),  # Match 1
    (ranked_teams[3], ranked_teams[4])   # Match 2
]

# Ergebnisse Woche 15
week15_results = []
for team1, team2 in week_15_matchups:
    lineup1 = get_best_lineup(team1, 15, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 15, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 15, weekly_data)
    points2 = get_actual_points(lineup2, 15, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week15_results.append({
        'Woche': 15,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_15 = [r['Sieger'] for r in week15_results]

# Woche 16 Matchups:
# Match 3: Winner Match 1 vs Seed 2
# Match 4: Winner Match 2 vs Seed 1
week_16_matchups = [
    (winners_15[0], ranked_teams[1]),  # gegen Seed 2
    (winners_15[1], ranked_teams[0])   # gegen Seed 1
]

week16_results = []
for team1, team2 in week_16_matchups:
    lineup1 = get_best_lineup(team1, 16, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 16, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 16, weekly_data)
    points2 = get_actual_points(lineup2, 16, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week16_results.append({
        'Woche': 16,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_16 = [r['Sieger'] for r in week16_results]

# Woche 17: Finale
week17_matchups = [(winners_16[0], winners_16[1])]

week17_results = []
for team1, team2 in week17_matchups:
    lineup1 = get_best_lineup(team1, 17, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 17, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 17, weekly_data)
    points2 = get_actual_points(lineup2, 17, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week17_results.append({
        'Woche': 17,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Ergebnisse zusammenführen
df_playoff_results = pd.DataFrame(week15_results + week16_results + week17_results)

# Optional an bestehende Ergebnisse anhängen:
df_results = pd.concat([df_results, df_playoff_results], ignore_index=True)

# Finale anzeigen
champion = df_playoff_results[df_playoff_results['Woche'] == 17]['Sieger'].values[0]
print(f"🏆 Der Champion ist: {champion}")
df_playoff_results

🏆 Der Champion ist: Team 4


,Woche,Team 1,Team 2,Punkte Team 1,Punkte Team 2,Sieger,Verlierer
0,15,Team 4,Team 11,111.04,64.76,Team 4,Team 11
1,15,Team 6,Team 9,104.64,69.60,Team 6,Team 9
2,16,Team 4,Team 7,92.32,87.24,Team 4,Team 7
3,16,Team 6,Team 1,74.88,123.64,Team 1,Team 6
4,17,Team 4,Team 1,127.58,110.28,Team 4,Team 1
